## Config

In [11]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-35-turbo-16k-dep-001"]
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    # "max_tokens": 8192 * 8,
}

## Construct Agents

In [12]:
urlBaseAddress = "http://192.168.0.103:5000/SamplesData"

ordersDataUrl=f"{urlBaseAddress}/Orders.csv"

customersDataUrl=f"{urlBaseAddress}/Customers.csv"

customersGroupsDataUrl=f"{urlBaseAddress}/CustomersGroups.csv"

productsDataUrl=f"{urlBaseAddress}/Products.csv"

activitiesDataUrl=f"{urlBaseAddress}/Activities.csv"

In [18]:
def is_termination_msg_func(x):
    r = x.get("content", "").rstrip().endswith("TERMINATE")
    print("===============", x)
    # r = True
    return r


# user_proxy = autogen.UserProxyAgent(
#     name="user_proxy",
#     # human_input_mode="TERMINATE",
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=10,
#     # is_termination_msg=lambda x: (x.get("content", "").rstrip().endswith("TERMINATE")),
#     is_termination_msg=is_termination_msg_func,
#     llm_config=llm_config,
#     code_execution_config={"work_dir": "00_awesome_poc"},
#     system_message="""
#     Execute the code written by others assistants and report the result.
#     Reply TERMINATE if the task has been solved at full satisfaction.
#     Otherwise, reply CONTINUE, or the reason why the task is not solved yet.
#     """
# )


user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config,
    code_execution_config= {"work_dir": "01_awesome_poc"},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''
    Planner. 
    Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
    The plan may involve an engineer who can write code and others don't write code.
    Explain the plan first. Be clear which step is performed by an engineer.
    ''',
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. 
    Wrap the code in a code block that specifies the script type. 
    The user can't modify your code. So do not suggest incomplete code which requires others to modify. 
    Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
    Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
    analyze the problem, revisit your assumption, 
    collect additional info you need, and think of a different approach to try.
''',
)

ordersAssistant = autogen.AssistantAgent(
    name="OrdersAssistant",
    llm_config=llm_config,
    # system_message=f"""
    # OrdersAssistant. 
    # 你负责告知 "Engineer": "Orders" 订单原始数据，从如下地址下载：{ordersDataUrl}，
    # 以及数据中的字段包括：OrderId,OrderNo,CustomerNo,Product,Price。
    # 你不用编写代码。
    # """
    system_message=f"""
     OrdersAssistant. 
     您负责编写 "Orders" 订单相关数据处理的 python/shell 代码辅助完成任务。
     "Orders"订单数据从: {ordersDataUrl} 下载，
     数据中包括字段：OrderId,OrderNo,CustomerNo,Product,Price
     你编写的代码包装在指定脚本类型的代码块中。
     用户无法修改您的代码。 所以不要建议不完整的代码，需要其他人修改。
     如果代码块不打算执行，则不要使用它。
     不要在一个响应中包含多个代码块。 不要要求其他人复制并粘贴结果。
     检查返回的执行结果。
     如果结果表明有错误，请修复错误并再次输出代码。
     建议使用完整代码，而不是部分代码或代码更改。
     如果错误无法修复或者代码执行成功后任务仍未解决，
     分析问题，重新审视你的假设，
     收集您需要的其他信息，并考虑尝试不同的方法。
    """

)

productsAssistant = autogen.AssistantAgent(
    name="ProductsAssistant",
    llm_config=llm_config,
    # system_message=f"""
    # ProductsAssistant. 
    # 你负责告知 "Engineer": "Orders" 订单原始数据，从如下地址下载：{productsDataUrl}，
    # 以及数据中的字段包括：Product,Unit,Stock,UnitPrice。
    # 你不用编写代码。
    # """
    system_message=f"""
    ProductsAssistant. 
    您负责编写 "Products" 订单相关数据处理的 python/shell 编写辅助完成任务。
    "Products"订单数据从: {productsDataUrl} 下载，
    数据中包括字段：Product,Unit,Stock,UnitPrice
    你编写的代码包装在指定脚本类型的代码块中。
    用户无法修改您的代码。 所以不要建议不完整的代码，需要其他人修改。
    如果代码块不打算执行，则不要使用它。
    不要在一个响应中包含多个代码块。 不要要求其他人复制并粘贴结果。
    检查返回的执行结果。
    如果结果表明有错误，请修复错误并再次输出代码。
    建议使用完整代码，而不是部分代码或代码更改。
    如果错误无法修复或者代码执行成功后任务仍未解决，
    分析问题，重新审视你的假设，
    收集您需要的其他信息，并考虑尝试不同的方法。
    """
)

groupchat = autogen.GroupChat(
    agents=[
    user_proxy, 
    # planner, 
    # engineer,
    # executor,
    ordersAssistant, 
    # productsAssistant
    ], 
    messages=[], max_round=20)
    
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [20]:
user_proxy.initiate_chat(
    manager,
#     message="""
# 获取最受欢迎的商品，然后查询该商品的库存量
# """,

#     message="""
# 获取最受欢迎的商品
# """,

message="""
获取销量最多商品
""",
)

user_proxy (to chat_manager):


获取销量最多商品


--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Direct communication would be more efficient.


KeyboardInterrupt: 